In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn as sk 

from matplotlib import style
style.use('dark_background')

In [2]:
train_data = pd.read_csv('train_data.csv')
train_data = train_data.drop(['ID','HealthServiceArea'], axis = 1)
train_data.head()

,Gender,Race,TypeOfAdmission,CCSProcedureCode,APRSeverityOfIllnessCode,PaymentTypology,BirthWeight,EmergencyDepartmentIndicator,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode,LengthOfStay
0,F,Other Race,Newborn,228,1,Medicaid,3700,N,2611,9227,1751,8951,45,1
1,M,Black/African American,Newborn,228,1,Medicaid,2900,N,3242,8966,3338,6409,34,1
2,M,Other Race,Newborn,220,1,Private Health Insurance,3200,N,3155,11381,4980,9323,45,1
3,F,Other Race,Newborn,0,1,Private Health Insurance,3300,N,3155,11381,5826,15680,59,1
4,F,Other Race,Newborn,228,1,Medicaid,2600,N,2611,9227,6000,14344,59,1


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59966 entries, 0 to 59965
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Gender                        59966 non-null  object
 1   Race                          59966 non-null  object
 2   TypeOfAdmission               59966 non-null  object
 3   CCSProcedureCode              59966 non-null  int64 
 4   APRSeverityOfIllnessCode      59966 non-null  int64 
 5   PaymentTypology               59966 non-null  object
 6   BirthWeight                   59966 non-null  int64 
 7   EmergencyDepartmentIndicator  59966 non-null  object
 8   AverageCostInCounty           59966 non-null  int64 
 9   AverageChargesInCounty        59966 non-null  int64 
 10  AverageCostInFacility         59966 non-null  int64 
 11  AverageChargesInFacility      59966 non-null  int64 
 12  AverageIncomeInZipCode        59966 non-null  int64 
 13  LengthOfStay    

In [4]:
train_data.describe()

,CCSProcedureCode,APRSeverityOfIllnessCode,BirthWeight,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode,LengthOfStay
count,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000
mean,155.404229,1.254594,3336.298903,2372.806690,7979.126922,2396.414318,7958.472668,59.811143,2.538405
std,89.541978,0.546207,446.244475,639.755096,3220.291347,1248.501189,3859.294711,21.470170,1.171246
min,-1.000000,1.000000,2500.000000,712.000000,1243.000000,457.000000,1120.000000,28.000000,1.000000
25%,115.000000,1.000000,3000.000000,2041.000000,4620.000000,1551.000000,4438.000000,45.000000,2.000000
50%,220.000000,1.000000,3300.000000,2533.000000,9227.000000,1967.000000,7945.000000,55.000000,2.000000
75%,228.000000,1.000000,3600.000000,2785.000000,10644.000000,2895.000000,11619.000000,74.000000,3.000000
max,231.000000,4.000000,7500.000000,3242.000000,11381.000000,8114.000000,18466.000000,115.000000,10.000000


In [5]:
X = train_data.drop('LengthOfStay', axis = 1)
Y = train_data['LengthOfStay']
Y_convert = Y.apply(lambda x: 0 if int(x) <= 3 else 1)

In [6]:
Y.value_counts()

2     25000
3     16000
1      8895
4      7504
5      1342
6       557
7       346
8       145
9        97
10       80
Name: LengthOfStay, dtype: int64

In [7]:
Y_convert.value_counts()

0    49895
1    10071
Name: LengthOfStay, dtype: int64

In [8]:
for col in X.columns:
    print(X[col].value_counts())
    print('---------------------------')

M    30978
F    28987
U        1
Name: Gender, dtype: int64
---------------------------
White                     32943
Other Race                18314
Black/African American     8183
Multi-racial                526
Name: Race, dtype: int64
---------------------------
Newborn      58741
Emergency      659
Urgent         412
Elective       154
Name: TypeOfAdmission, dtype: int64
---------------------------
 228    19886
 115    13628
 0      11189
 220    10773
 231     2981
-1        769
 216      740
Name: CCSProcedureCode, dtype: int64
---------------------------
1    47953
2     8760
3     3252
4        1
Name: APRSeverityOfIllnessCode, dtype: int64
---------------------------
Medicaid                     28723
Private Health Insurance     15608
Blue Cross/Blue Shield       12073
Self-Pay                      1984
Federal/State/Local/VA         849
Managed Care, Unspecified      545
Miscellaneous/Other            118
Medicare                        44
Unknown                        

## Dummy Encode

In [15]:
# Select columns for one-hot encoding
# CSS Procedure Code not sure
encode_col = ['Gender','Race','TypeOfAdmission','PaymentTypology','EmergencyDepartmentIndicator']
X_copy = X.copy()


for col in encode_col:
    # create new dataframe for each type of encode_column
    dummiesDF = pd.get_dummies(X_copy[col])
    # merge the new dataframe into the orginal
    X_copy = pd.concat([X_copy, dummiesDF], axis='columns')
    # Drop the original column after encoding
    if col == 'Gender':
        X_copy = X_copy.drop([col,'U'], axis = 1)
    else:
        X_copy = X_copy.drop(col, axis = 1)

X_copy.columns


Index(['CCSProcedureCode', 'APRSeverityOfIllnessCode', 'BirthWeight',
       'AverageCostInCounty', 'AverageChargesInCounty',
       'AverageCostInFacility', 'AverageChargesInFacility',
       'AverageIncomeInZipCode', 'F', 'M', 'Black/African American',
       'Multi-racial', 'Other Race', 'White', 'Elective', 'Emergency',
       'Newborn', 'Urgent', 'Blue Cross/Blue Shield', 'Federal/State/Local/VA',
       'Managed Care, Unspecified', 'Medicaid', 'Medicare',
       'Miscellaneous/Other', 'Private Health Insurance', 'Self-Pay',
       'Unknown', 'N', 'Y'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X_copy,Y,test_size = 0.3, stratify = Y)

print(train_X.shape)
print(test_X.shape)
print(train_Y.shape)
print(test_Y.shape)

(41976, 29)
(17990, 29)
(41976,)
(17990,)


In [19]:
# Reset index after spliting

def df_reset_index(x):
    return x.reset_index().drop('index', axis = 1)

train_X = df_reset_index(train_X)
test_X = df_reset_index(test_X)
train_Y = df_reset_index(train_Y)
test_Y = df_reset_index(test_Y)

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale = scaler.transform(test_X)
train_Y_ravel = train_Y.values.ravel()

# Không transform data

In [25]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state = 0)
log_reg.fit(train_X_scale, train_Y_ravel)


C:\Users\luong\anaconda3\envs\Basic Tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)